In [1]:
import psycopg2 # type: ignore
import matplotlib.pyplot as plt # type: ignore
import pandas as pd # type: ignore
from tqdm import tqdm # type: ignore
import numpy as np # type: ignore
import pandas as pd # type: ignore
import plotly.graph_objects as go # type: ignore
config = {
    "host": "10.188.200.16",
    "dbname": "tw",
    "user": "jimmy",
    "password": "123123"
}
conn = psycopg2.connect(**config)
cursor = conn.cursor()

# tw_maincode = """
# select * from price_fund where da >='2019-01-01' and da <='2020-01-01';
# """
# cursor.execute(tw_maincode)
# conn.commit()
# res = cursor.fetchall()
# columns = [desc[0] for desc in cursor.description]

In [2]:
# price
sqlStr = """
SELECT DISTINCT p.da, p.nav, p.code, m.fund_type_code
FROM maincode_fund AS m
LEFT JOIN price_fund AS p ON p.code = m.code
WHERE p.da >= '2016-01-01' and m.year = '2025'
ORDER BY p.code, p.da DESC;
"""
cursor.execute(sqlStr)
res = cursor.fetchall()
df = pd.DataFrame(res)
df.columns = ["da", "nav", "code", "fund_type"]
fundtype_list = list(set(df.fund_type))

# fund type to be of stocks
sqlStr = """
SELECT distinct fund_type_code
FROM maincode_fund
WHERE year = '2025' 
AND fund_type LIKE '%股票%'
ORDER BY fund_type_code;
"""
cursor.execute(sqlStr)
conn.commit()
res = cursor.fetchall()
fundtype_list_stock = [i[0] for i in res]

# fund type in Mandarin
sqlStr = """
SELECT distinct fund_type, fund_type_code
FROM maincode_fund
WHERE year = '2025';
"""
cursor.execute(sqlStr)
conn.commit()
res = cursor.fetchall()
fundtype_list_mandarin = {i[0]: i[1] for i in res}

In [4]:
# benchmark_list = [
#     {{'組合型–跨國投資(債券型)': 'AE22'}: {'彭博巴克萊全球綜合債券指數': 'AE22'}},
#     {{'跨國投資多重資產型': 'AJ2'}: {'60% MSCI世界指數 + 40% 彭博巴克萊全球綜合債券指數': 'AJ2'}},
#     {{'國內ETF連結基金': 'AK1'}: {'台灣50指數 (TW50)': 'AK1'}},
#     {{'國內投資指數股票型_槓桿型/反向型': 'AH13'}: {'台灣加權指數 (TAIEX)': 'AH13'}},
#     {{'保本型': 'AF'}: {'台灣三個月國庫券指數': 'AF'}},
#     {{'固定收益型–跨國投資(一般債券)': 'AC21'}: {'彭博巴克萊全球綜合債券指數': 'AC21'}},
#     {{'跨國投資組合型_股票型': 'AE21'}: {'MSCI世界指數': 'AE21'}},
#     {{'國內投資平衡型': 'AB1'}: {'60% 台灣加權指數 + 40% 台灣債券指數': 'AB1'}},
#     {{'跨國投資指數股票型_不動產證券化/其他': 'AH25'}: {'FTSE EPRA/NAREIT全球不動產指數': 'AH25'}},
#     {{'跨國投資指數股票型_槓桿型/反向型_股票': 'AH23'}: {'MSCI世界指數': 'AH23'}},
#     {{'跨國投資平衡型': 'AB2'}: {'60% MSCI世界指數 + 40% 彭博巴克萊全球綜合債券指數': 'AB2'}},
#     {{'國內投資組合型': 'AE1'}: {'台灣加權指數 (TAIEX)': 'AE1'}},
#     {{'跨國投資股票型': 'AA2'}: {'MSCI世界指數': 'AA2'}},
#     {{'國內投資指數股票型_股票型': 'AH11'}: {'台灣加權指數 (TAIEX)': 'AH11'}},
#     {{'跨國投資組合型_其他': 'AE24'}: {'MSCI全球指數 (MSCI ACWI)': 'AE24'}},
#     {{'非投資等級債券型': 'AC23'}: {'彭博全球高收益債券指數': 'AC23'}},
#     {{'組合型–跨國投資(平衡型)': 'AE23'}: {'60% MSCI世界指數 + 40% 彭博巴克萊全球綜合債券指數': 'AE23'}},
#     {{'國內投資固定收益一般債券型': 'AC12'}: {'台灣債券指數': 'AC12'}},
#     {{'跨國投資組合型_平衡型': 'AE23'}: {'60% MSCI世界指數 + 40% 彭博巴克萊全球綜合債券指數': 'AE23'}},
#     {{'跨國投資組合型_債券型': 'AE22'}: {'彭博巴克萊全球綜合債券指數': 'AE22'}},
#     {{'固定收益型–跨國投資(高收益債券)': 'AC23'}: {'彭博全球高收益債券指數': 'AC23'}},
#     {{'股票型–國內投資': 'AA1'}: {'台灣加權指數 (TAIEX)': 'AA1'}},
#     {{'跨國投資固定收益一般債券型': 'AC21'}: {'彭博巴克萊全球綜合債券指數': 'AC21'}},
#     {{'跨國投資指數股票型_槓桿型/反向型_債券': 'AH24'}: {'MSCI世界指數': 'AH24'}},
#     {{'國內投資貨幣市場基金': 'AD1'}: {'台灣三個月國庫券指數': 'AD1'}},
#     {{'不動產證券化型': 'AG'}: {'FTSE EPRA/NAREIT全球不動產指數': 'AG'}},
#     {{'國內投資多重資產型': 'AJ1'}: {'60% 台灣加權指數 + 40% 台灣債券指數': 'AJ1'}},
#     {{'跨國投資指數型': 'AI2'}: {'MSCI世界指數': 'AI2'}},
#     {{'跨國投資指數股票型_債券型': 'AH22'}: {'MSCI世界指數': 'AH22'}},
#     {{'國內投資股票型': 'AA1'}: {'台灣加權指數 (TAIEX)': 'AA1'}},
#     {{'跨國投資指數股票型_股票型': 'AH21'}: {'MSCI世界指數': 'AH21'}},
#     {{'債券股票平衡型–跨國投資': 'AB2'}: {'60% MSCI世界指數 + 40% 彭博巴克萊全球綜合債券指數': 'AB2'}},
#     {{'股票型–跨國投資': 'AA2'}: {'MSCI世界指數': 'AA2'}},
#     {{'跨國投資貨幣市場基金': 'AD2'}: {'彭博全球貨幣市場指數': 'AD2'}},
#     {{'國內投資指數型': 'AI1'}: {'台灣加權指數 (TAIEX)': 'AI1'}},
#     {{'不動產證券化': 'AG'}: {'FTSE EPRA/NAREIT全球不動產指數': 'AG'}}
# ]


In [ ]:
import plotly.graph_objects as go # type: ignore
from plotly.subplots import make_subplots
type_ = "AA1"
tdf = df[df['fund_type'] == type_].pivot(values='nav', index='da', columns='code')  
tdf = tdf.ffill()
pct_change = tdf.pct_change()
pct_change[(pct_change > 0.1) | (pct_change < -0.1)] = 0
s = pct_change.cumsum().fillna(0)

pct_change = pct_change.iloc[:, :30]
init = 20
dates = pct_change.index[init:] 
turbulences = [0.0] * len(pct_change)
for i in range(len(dates)):
    pct_1 = pct_change.iloc[i:init+i, :]
    mean = np.array(pct_1.mean())
    cov = np.array(pct_1.cov())
    pct_current = np.array(pct_1.iloc[-1])
    
    turbulence = (pct_current - mean) @ cov @ (pct_current - mean).T
    turbulences[init+i] = turbulence * 10000
    
turbulence_df = pd.DataFrame(turbulences, columns=["turbulence"])
turbulence_df.index = pct_change.index
mm = pct_change.mean(axis=1)
for i in range(len(pct_change.columns)):
    pct_change.iloc[:, i] = pct_change.iloc[:, i] - mm
cumsum = pct_change.cumsum()
cumsum_shift1 = cumsum.shift(-1)

fig = go.Figure();

fig = make_subplots(
    rows=2, cols=1,  
    shared_xaxes=True,  
    vertical_spacing=0.1,  
    subplot_titles=["Cumulative Return", "Another Plot"],  
    row_heights=[0.7, 0.3]  
)
for i in s.columns[:]:
    temp = s[i]
    fig.add_trace(go.Scatter(
        x=temp.index,
        y=temp.values,
        name=i,
        opacity=0.8,
        yaxis="y1"  # Assigning to the first y-axis
    ), row=1, col=1)

fig.add_trace(go.Bar(
    x=turbulence_df.index,
    y=turbulence_df["turbulence"],
    name="Turbulence",
    marker=dict(color="black"),
    opacity=1,
), row=2, col=1)


fig.show()

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

def bt(df, type_='AH21'):
    tdf = df[df['fund_type'] == type_].pivot(values='nav', index='da', columns='code')
    tdf = tdf.ffill()
    pct_change = tdf.pct_change()
    pct_change[(pct_change > 0.1) | (pct_change < -0.1)] = 0
    pct_change.index = pd.to_datetime(pct_change.index)
    pct_change = pct_change.resample("W").sum()
    pct_change = pct_change.iloc[:, :]
    print(len(pct_change))
    init = 20
    dates = pct_change.index[init:] 
    turbulences = [0.0] * len(pct_change)
    for i in range(len(dates)):
        pct_1 = pct_change.iloc[i:init+i, :]
        mean = np.array(pct_1.mean())
        cov = np.array(pct_1.cov())
        pct_current = np.array(pct_1.iloc[-1])
        
        turbulence = (pct_current - mean) @ cov @ (pct_current - mean).T
        turbulences[init+i] = turbulence * 10000
        
    turbulence_df = pd.DataFrame(turbulences, columns=["turbulence"])
    turbulence_df.index = pct_change.index
    mm = pct_change.mean(axis=1)
    for i in range(len(pct_change.columns)):
        pct_change.iloc[:, i] = pct_change.iloc[:, i] - mm

    position = []
    holding_history = {}

    for date, row in turbulence_df.iterrows():
        t = row.iloc[0]
        if t > 5:
            k = pct_change.loc[:date].iloc[:, :]
            top_five = k.cumsum().iloc[-1].nlargest(1).index
            if position != list(top_five):
                position = list(top_five)
                holding_history[date] = position

    pct_change_shift1 = pct_change.shift(-1)

    holding_df = pd.DataFrame(list(holding_history.items()), columns=['Date', 'Code']).set_index('Date')
    hhdf = pd.DataFrame(index=pct_change_shift1.index)
    hhdf['code'] = holding_df
    hhdf.index.name = "Date"
    hhdf = hhdf.ffill()

    hhdf1 = hhdf.join(pct_change_shift1, on='Date')
    hhdf1 = hhdf1.fillna(0)
    hhdf1['pct_change'] = hhdf1.apply(lambda x: 0 if x.iloc[0] == 0 else x[x.iloc[0]].mean(), axis=1)
    hhdf1['pct_change'] = hhdf1['pct_change'].cumsum()
    fig = go.Figure()

    # Adding turbulence bar plot
    fig.add_trace(go.Bar(
        x=turbulence_df.index,
        y=turbulence_df["turbulence"],
        name="Turbulence",
        marker=dict(color="blue"),
        opacity=0.7,
        yaxis="y1"  # Assigning to the first y-axis
    ))


    fig.add_trace(go.Scatter(
        x=hhdf1.index,
        y=hhdf1["pct_change"],
        name="Pct Change",
        mode="lines",
        line=dict(color="green"),
        yaxis="y2"
    ))
    
    # # beck = beck.loc["2020-01-01":].pct_change().cumsum()
    # beck = beck.loc[:].pct_change().cumsum()
    # beck.columns = ['pct_change']
    # fig.add_trace(go.Scatter(
    #     x=beck.index,
    #     y=beck["pct_change"],
    #     name="Pct bm",
    #     mode="lines",
    #     line=dict(color="orange"),
    #     yaxis="y2"
    # ))
    fig.update_layout(
        title="Turbulence and Percent Change Over Time",
        xaxis=dict(title="Date"),
        yaxis=dict(
            title="Turbulence",
            titlefont=dict(color="blue"),
            tickfont=dict(color="blue"),
        ),
        yaxis2=dict(
            title="Pct Change",
            titlefont=dict(color="orange"),
            tickfont=dict(color="orange"),
            overlaying="y",  # Overlaying on the same x-axis
            side="right"  # Placed on the right side
        ),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        template="plotly_white"
    )

    fig.show()
    return holding_df
holding_df = bt(df, type_="AH21")

In [8]:
def easy_breakout(df, beck, top_n=1, type_="AE21"):
   
    tdf = df[df['fund_type'] == type_].pivot(values='nav', index='da', columns='code')  
    tdf = tdf.ffill().fillna(0)
    
    pct_change = tdf.pct_change()
    pct_change[(pct_change > 0.1) | (pct_change < -0.1)] = 0 
   
    holdings = []
    cumulative_returns = pd.DataFrame(index=tdf.index)  

   
    for i in range(1, len(tdf)):
        cumulative_returns = cumulative_returns.shift(1).fillna(0) + pct_change
        best_stocks = cumulative_returns.iloc[i].nlargest(top_n).index 
        holdings.append(list(best_stocks)) 
    
    i = 20
    pct_change = pct_change.shift(i)
    pct_change['Best_Holding'] = [None] + holdings
    holds = ""
    holdings_dict = {}
    
    for index, row in pct_change[['Best_Holding']].iterrows():
        if row['Best_Holding'] == None: continue
        current_holdings = set(row['Best_Holding']) 
        if holds != current_holdings:
            holdings_dict[index] = current_holdings 
            holds = current_holdings

    pct_change = pct_change.iloc[2+i:, :]
    
        
    pct_change['ret'] = pct_change.apply(lambda row: row[row['Best_Holding']].mean(), axis=1)
    
   
    fig = go.Figure()
   
    fig.add_trace(go.Scatter(
        x=pct_change.index,
        y=pct_change['ret'].cumsum(),
        mode="lines",
        line=dict(color="green"),
        yaxis="y1",
        name="Strategy Return"
    ))

   
    beck = beck.loc[:].pct_change().cumsum()
    beck.columns = ['pct_change']
    fig.add_trace(go.Scatter(
        x=beck.index,
        y=beck["pct_change"],
        name="Benchmark",
        mode="lines",
        line=dict(color="orange"),
        yaxis="y1"
    ))
    
   
    fig.update_layout(
        title="Strategy and Benchmark Performance",
        xaxis=dict(title="Date"),
        yaxis=dict(
            title="Strategy Return",
            titlefont=dict(color="blue"),
            tickfont=dict(color="blue"),
        ),
        yaxis2=dict(
            title="Benchmark",
            titlefont=dict(color="orange"),
            tickfont=dict(color="orange"),
            overlaying="y", 
            side="right" 
        ),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        template="plotly_white"
    )

    fig.show()
    return holdings_dict


config = {
    "host": "10.188.200.16",
    "dbname": "tw",
    "user": "jimmy",
    "password": "123123"
}
conn = psycopg2.connect(**config)
cursor = conn.cursor()

sqlStr = """
select da, cl from price where code = '0050 TT Equity' and da >= '2016-01-01'
order by da asc;
"""
cursor.execute(sqlStr)
conn.commit()
res = cursor.fetchall()
beck = pd.DataFrame(res, columns=['da', 'cl'])
beck.set_index('da', inplace=True)
holdings_dict = easy_breakout(df, beck, top_n=3, type_='AA2')


In [ ]:
from numpy.linalg import LinAlgError #type: ignore
def calculate_turbulence(holding_df):
    for i in range(len(holding_df)):
        temp = holding_df.iloc[i:init+i, :]
        index = temp.index[-1]
        temp_arr = np.array(temp)
        m = np.mean(temp_arr, axis=0)
        c = temp.cov()
        try:
            turbulence = m.T @ np.linalg.inv(c) @ m
            turbulences[index] = turbulence
        except LinAlgError:
            turbulences[index] = 0
            print(1)
    return pd.DataFrame([turbulences]).T
iter = 0
prev = None
holding_dict = {}
for da, holding in holdings_dict.items():
    if not prev: holding_dict[da] = holding; prev = holding;
    # test whether modify position or not
    else:
        prev_holding = pct_change[list(prev)].loc[:da]
        curr_holding = pct_change[list(holding)].loc[:da]
        prev_turbulence = calculate_turbulence(prev_holding)
        curr_turbulence = calculate_turbulence(curr_holding)
        break
init = 20
turbulences = {}

for i in range(len(temp_df)):
    temp = temp_df.iloc[i:init+i, :]
    index = temp.index[-1]
    temp_arr = np.array(temp)
    m = np.mean(temp_arr, axis=0)
    c = temp.cov()
    try:
        turbulence = m.T @ np.linalg.inv(c) @ m
        turbulences[index] = turbulence
    except LinAlgError:
        turbulences[index] = 0
        print(1)